# INNER JOIN with Pandas

### 렉처 중간에 예제 문제가 있습니다. 풀면서 따라와주세요!

In [2]:
import pandas as pd

# 데이터 읽기

In [3]:
orders = pd.read_csv('https://raw.githubusercontent.com/dataitgirls3/Data/master/Orders.csv')
orderDetails = pd.read_csv('https://raw.githubusercontent.com/dataitgirls3/Data/master/OrderDetails.csv')
products = pd.read_csv('https://raw.githubusercontent.com/dataitgirls3/Data/master/Products.csv')
customers = pd.read_csv('https://raw.githubusercontent.com/dataitgirls3/Data/master/Customers.csv')
categories = pd.read_csv('https://raw.githubusercontent.com/dataitgirls3/Data/master/Categories.csv')
suppliers = pd.read_csv('https://raw.githubusercontent.com/dataitgirls3/Data/master/Suppliers.csv')
shippers = pd.read_csv('https://raw.githubusercontent.com/dataitgirls3/Data/master/Shippers.csv')
employees = pd.read_csv('https://raw.githubusercontent.com/dataitgirls3/Data/master/Employees.csv')

# 데이터 확인하기

In [4]:
print('columns in Orders: ' + str(list(orders.columns)))
print('columns in OrderDetails: ' + str(list(orderDetails.columns)))
print('columns in Products: ' + str(list(products.columns)))

columns in Orders: ['OrderID', 'CustomerID', 'EmployeeID', 'OrderDate', 'ShipperID']
columns in OrderDetails: ['OrderDetailID', 'OrderID', 'ProductID', 'Quantity']
columns in Products: ['ProductID', 'ProductName', 'SupplierID', 'CategoryID', 'Unit', 'Price']


In [5]:
print('총 ' + str(len(orders)) + '건의 주문Order 데이터')
print('총 ' + str(len(orderDetails)) + '건의 주문상세OrderDetails 데이터')

총 196건의 주문Order 데이터
총 518건의 주문상세OrderDetails 데이터


Order 1 : OrderDetails N

#### 테이블 관계
- 1:1 관계
- 1:N 관계
- M:N 관계 → 지양


In [8]:
orders.head(2)

,OrderID,CustomerID,EmployeeID,OrderDate,ShipperID
0,10248,90,5,1996-07-04,3
1,10249,81,6,1996-07-05,1


In [9]:
orderDetails.head(2)

,OrderDetailID,OrderID,ProductID,Quantity
0,1,10248,11,12
1,2,10248,42,10


In [10]:
products.head(2)

,ProductID,ProductName,SupplierID,CategoryID,Unit,Price
0,1,Chais,1,1,10 boxes x 20 bags,18.0
1,2,Chang,1,1,24 - 12 oz bottles,19.0


# 분석의 목표
orders -> orderDetails -> products 를 묶어서 order 당 몇 개의 상품을 주문하는지, 종류는 얼마나 다양한지, 총 가격은 얼마인지 등을 구해보자.

## Order 한 개에 얼마나 다양한 물건들이 들어있는가?
Order 한 개 당 몇 개의 ProductID를 가지고 있는지 세보라.

In [11]:
'''
SELECT *
FROM Orders AS o
     INNER JOIN OrderDetails AS od ON o.OrderID = od.OrderID
'''

order_orderDetails = orders.merge(orderDetails, how='inner', on='OrderID')

In [12]:
order_orderDetails.head(2)

,OrderID,CustomerID,EmployeeID,OrderDate,ShipperID,OrderDetailID,ProductID,Quantity
0,10248,90,5,1996-07-04,3,1,11,12
1,10248,90,5,1996-07-04,3,2,42,10


In [13]:
'''
SELECT o.OrderID, COUNT(*)
FROM Orders AS o
     INNER JOIN OrderDetails AS od ON o.orderID = od.OrderID
GROUP BY o.OrderID
'''

products_per_order = order_orderDetails.groupby('OrderID').size()
products_per_order

OrderID
10248    3
10249    2
10250    3
10251    3
10252    3
10253    3
10254    3
10255    4
10256    2
10257    3
10258    3
10259    2
10260    4
10261    2
10262    3
10263    4
10264    2
10265    2
10266    1
10267    3
10268    2
10269    2
10270    2
10271    1
10272    3
10273    5
10274    2
10275    2
10276    2
10277    2
        ..
10414    2
10415    2
10416    3
10417    4
10418    4
10419    2
10420    4
10421    4
10422    1
10423    2
10424    3
10425    2
10426    2
10427    1
10428    1
10429    2
10430    4
10431    3
10432    2
10433    1
10434    2
10435    3
10436    4
10437    1
10438    3
10439    4
10440    4
10441    1
10442    3
10443    2
Length: 196, dtype: int64

In [14]:
# 평균적으로 1오더 당 3가지의 Product가 들어있다.
products_per_order.mean()

2.642857142857143

## Order 한 개에 총 몇 개의 물건이 들어있는가?

In [16]:
'''
SELECT o.OrderID
     , SUM(od.Quantity)
FROM Orders AS o
     INNER JOIN OrderDetails AS od ON o.OrderID = od.OrderID
GROUP BY o.OrderID
'''

num_items_per_order = order_orderDetails.groupby('OrderID')['Quantity'].sum()
num_items_per_order

OrderID
10248     27
10249     49
10250     60
10251     41
10252    105
10253    102
10254     57
10255    110
10256     27
10257     46
10258    121
10259     11
10260    102
10261     40
10262     29
10263    184
10264     60
10265     50
10266     12
10267    135
10268     14
10269     80
10270     55
10271     24
10272     70
10273    152
10274     27
10275     18
10276     25
10277     32
        ... 
10414     68
10415     22
10416     50
10417    123
10418    146
10419     80
10420     50
10421     59
10422      2
10423     34
10424    139
10425     30
10426     12
10427     35
10428     20
10429     75
10430    195
10431    130
10432     50
10433     28
10434     24
10435     32
10436     99
10437     15
10438     50
10439     67
10440    208
10441     50
10442    170
10443     18
Name: Quantity, Length: 196, dtype: int64

In [17]:
# 한 주문 당 평균적으로 65.02개의 물건이 들어있다.
num_items_per_order.mean()

65.01530612244898

# Quiz. 65 개 이상의 물건이 들어있는 주문의 OrderID 를 구해주세요.

In [21]:
order_all = orders.merge(orderDetails, how = 'inner', on='OrderID')
order_all.head()

,OrderID,CustomerID,EmployeeID,OrderDate,ShipperID,OrderDetailID,ProductID,Quantity
0,10248,90,5,1996-07-04,3,1,11,12
1,10248,90,5,1996-07-04,3,2,42,10
2,10248,90,5,1996-07-04,3,3,72,5
3,10249,81,6,1996-07-05,1,4,14,9
4,10249,81,6,1996-07-05,1,5,51,40


In [33]:
'''
SELECT OrderID, SUM(Quantity)
FROM ORDER_DETAILS
GROUP BY OrderID

'''

order_product_count = order_all.groupby('OrderID')['Quantity'].sum()

In [36]:
order_product_count = order_product_count.reset_index()

In [40]:
order_product_count[order_product_count['Quantity'] >= 65].head()

,OrderID,Quantity
4,10252,105
5,10253,102
7,10255,110
10,10258,121
12,10260,102


## 주문마다 총 가격을 구해주세요.

In [55]:
# SQL로는 이렇게 쓸 수 있겠죠!

'''
SELECT o.OrderID
     , SUM(od.Quantity*p.Price)
FROM Orders AS o
     INNER JOIN OrderDetails AS od ON o.OrderID = od.OrderID
     INNER JOIN Products AS p ON od.ProductID = p.ProductID
GROUP BY o.OrderID
'''

'''
SELECT od.OrderID
     , SUM(od.Quantity*p.Price)
FROM OrderDetails AS od
INNER JOIN Prodeucts AS p ON od.ProductID = p.ProductID
GROUP BY od.OrderID
'''

'\nSELECT o.OrderID\n     , SUM(od.Quantity*p.Price)\nFROM Orders AS o\n     INNER JOIN OrderDetails AS od ON o.OrderID = od.OrderID\n     INNER JOIN Products AS p ON od.ProductID = p.ProductID\nGROUP BY o.OrderID\n'

# Quiz. 주문별 주문 금액을 구하세요.
1. OrderDetails, Products MERGE
2. Quantity * Price 곱한 필드를 만들어 주세요
3. OrderID 별로 GROUP BY SUM() 해주세요 

In [55]:
# Merge 할 때 On 하는 Column 이름이 각자 다를 때는, left_on = '' right_on = '' 에 각각 넣어준다
# column 이 아니라 index 로 조인하고 싶을 때는, on 대신 left_index=True, right_index=True 를 넣는다
order_details_products = orderDetails.merge(products, how = 'inner', on = 'ProductID')
order_details_products.head()

,OrderDetailID,OrderID,ProductID,Quantity,ProductName,SupplierID,CategoryID,Unit,Price
0,1,10248,11,12,Queso Cabrales,5,4,1 kg pkg.,21.0
1,130,10296,11,12,Queso Cabrales,5,4,1 kg pkg.,21.0
2,211,10327,11,50,Queso Cabrales,5,4,1 kg pkg.,21.0
3,281,10353,11,12,Queso Cabrales,5,4,1 kg pkg.,21.0
4,314,10365,11,24,Queso Cabrales,5,4,1 kg pkg.,21.0


In [56]:
order_details_products['Quantity * Price'] = order_details_products['Quantity'] * order_details_products['Price']
order_details_products.head()

,OrderDetailID,OrderID,ProductID,Quantity,ProductName,SupplierID,CategoryID,Unit,Price,Quantity * Price
0,1,10248,11,12,Queso Cabrales,5,4,1 kg pkg.,21.0,252.0
1,130,10296,11,12,Queso Cabrales,5,4,1 kg pkg.,21.0,252.0
2,211,10327,11,50,Queso Cabrales,5,4,1 kg pkg.,21.0,1050.0
3,281,10353,11,12,Queso Cabrales,5,4,1 kg pkg.,21.0,252.0
4,314,10365,11,24,Queso Cabrales,5,4,1 kg pkg.,21.0,504.0


In [58]:
d = order_details_products.groupby('OrderID')['Quantity * Price'].sum()
d.head()

OrderID
10248     566.00
10249    2329.25
10250    2267.25
10251     839.50
10252    4662.50
Name: Quantity * Price, dtype: float64

In [66]:
d.sort_values(ascending=False).head()

OrderID
10372    15353.60
10424    14366.50
10417    14104.00
10353    13427.00
10360     9244.25
Name: Quantity * Price, dtype: float64

In [64]:
# DataFrame 은 Columns 이름을 by='Column_name' 넣어준다
d.reset_index().sort_values(by='Quantity * Price', ascending=False).head()

,OrderID,Quantity * Price
124,10372,15353.60
176,10424,14366.50
169,10417,14104.00
105,10353,13427.00
112,10360,9244.25


In [41]:
# 변수명 이렇게 하시면 나중에 헷갈리고 힘들어요... 저는 강의니까 그냥 이렇게 하께요 (변명)
order_all = order_orderDetails.merge(products, how='inner', on='ProductID')
order_all.head(2)

,OrderID,CustomerID,EmployeeID,OrderDate,ShipperID,OrderDetailID,ProductID,Quantity,ProductName,SupplierID,CategoryID,Unit,Price
0,10248,90,5,1996-07-04,3,1,11,12,Queso Cabrales,5,4,1 kg pkg.,21.0
1,10296,46,6,1996-09-03,1,130,11,12,Queso Cabrales,5,4,1 kg pkg.,21.0


In [70]:
# 컬럼이 너무 많으니까 일단 지금 관심 있는 것만 추려볼게요
order_all_part = order_all[['OrderID', 'OrderDate', 'Quantity', 'ProductID', 'CategoryID', 'Price']]

In [73]:
total_price = order_all_part['Price'] * order_all_part['Quantity']
order_all_part['TotalPricePerProduct'] = total_price

/usr/local/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [76]:
price_per_order = order_all_part.groupby('OrderID')['TotalPricePerProduct'].sum()
price_per_order

OrderID
10248     566.00
10249    2329.25
10250    2267.25
10251     839.50
10252    4662.50
          ...   
10439    1348.70
10440    7246.01
10441    2195.00
10442    2246.00
10443     673.20
Name: TotalPricePerProduct, Length: 196, dtype: float64

## 총 가격이 1000불 이상인 주문만 뽑아주세요.

In [77]:
# SQL로는 이렇게 쓸 수 있겠죠!

'''
SELECT o.OrderID
     , SUM(od.Quantity*p.Price) AS PricePerOrder
FROM Orders AS o
     INNER JOIN OrderDetails AS od ON o.OrderID = od.OrderID
     INNER JOIN Products AS p ON od.ProductID = p.ProductID
GROUP BY o.OrderID
HAVING PricePerOrder >= 1000
'''

'\nSELECT o.OrderID\n     , SUM(od.Quantity*p.Price) AS PricePerOrder\nFROM Orders AS o\n     INNER JOIN OrderDetails AS od ON o.OrderID = od.OrderID\n     INNER JOIN Products AS p ON od.ProductID = p.ProductID\nGROUP BY o.OrderID\nHAVING PricePerOrder >= 1000\n'

In [82]:
price_per_order[price_per_order >= 1000]

OrderID
10249    2329.25
10250    2267.25
10252    4662.50
10253    1806.00
10255    3115.75
          ...   
10436    2763.50
10439    1348.70
10440    7246.01
10441    2195.00
10442    2246.00
Name: TotalPricePerProduct, Length: 118, dtype: float64

### Q1. Products의 CategoryID와 Categories 테이블을 이용해서, 어떤 카테고리의 상품이 가장 많이 주문되었는지 구하세요.

In [28]:
ca = orders.merge(orderDetails, on='OrderID', how='inner').merge(products, on='ProductID')
ca.sample()
# ca

,OrderID,CustomerID,EmployeeID,OrderDate,ShipperID,OrderDetailID,ProductID,Quantity,ProductName,SupplierID,CategoryID,Unit,Price
418,10436,7,3,1997-02-05,2,497,46,5,Spegesild,21,8,4 - 450 g glasses,12.0


In [29]:
# ca.groupby('CategoryID')['CategoryName'].size()
ca['CategoryID'].value_counts()

4    100
1     93
3     84
8     67
6     50
2     49
5     42
7     33
Name: CategoryID, dtype: int64

### Q2. Orders 테이블의 OrderDate를 이용해, 어떤 날에 주문이 가장 많았는지 알려주세요.

In [45]:
order_count = orders['OrderDate'].value_counts()

In [50]:
order_count[order_count == order_count.max()].index

Index(['1997-01-16', '1996-08-14', '1996-12-25', '1996-12-23', '1997-01-27',
       '1996-12-05', '1997-01-30', '1997-02-10', '1996-10-29', '1996-12-03',
       '1996-12-16', '1997-01-03', '1996-11-22', '1996-10-03', '1997-02-03',
       '1997-01-23', '1996-09-09', '1996-12-09', '1996-12-18', '1997-01-21',
       '1996-12-12', '1996-07-08', '1996-11-11', '1997-02-05', '1996-09-20',
       '1997-01-10', '1996-08-27', '1997-01-01', '1996-11-28', '1997-01-14',
       '1997-01-07', '1996-07-19', '1996-08-01', '1996-11-26', '1996-10-16',
       '1996-12-27'],
      dtype='object')

### Q3. Orders 테이블의 OrderDate를 이용해, 어떤 날에 매출금액이 가장 높았는지 알려주세요.

In [59]:
od = orders.merge(orderDetails, on='OrderID', how='inner').merge(products, on='ProductID', how='inner')
od.sample()

,OrderID,CustomerID,EmployeeID,OrderDate,ShipperID,OrderDetailID,ProductID,Quantity,ProductName,SupplierID,CategoryID,Unit,Price
60,10401,65,1,1997-01-01,1,411,65,20,Louisiana Fiery Hot Pepper Sauce,2,2,32 - 8 oz bottles,21.05


In [60]:
od['Total_Price'] = od['Quantity'] * od['Price']
od

,OrderID,CustomerID,EmployeeID,OrderDate,ShipperID,OrderDetailID,ProductID,Quantity,ProductName,SupplierID,CategoryID,Unit,Price,Total_Price
0,10248,90,5,1996-07-04,3,1,11,12,Queso Cabrales,5,4,1 kg pkg.,21.00,252.00
1,10296,46,6,1996-09-03,1,130,11,12,Queso Cabrales,5,4,1 kg pkg.,21.00,252.00
2,10327,24,2,1996-10-11,1,211,11,50,Queso Cabrales,5,4,1 kg pkg.,21.00,1050.00
3,10353,59,7,1996-11-13,3,281,11,12,Queso Cabrales,5,4,1 kg pkg.,21.00,252.00
4,10365,3,3,1996-11-27,2,314,11,24,Queso Cabrales,5,4,1 kg pkg.,21.00,504.00
5,10407,56,2,1997-01-07,2,426,11,30,Queso Cabrales,5,4,1 kg pkg.,21.00,630.00
6,10434,24,3,1997-02-03,2,492,11,6,Queso Cabrales,5,4,1 kg pkg.,21.00,126.00
7,10442,20,3,1997-02-11,2,514,11,30,Queso Cabrales,5,4,1 kg pkg.,21.00,630.00
8,10443,66,8,1997-02-12,1,517,11,6,Queso Cabrales,5,4,1 kg pkg.,21.00,126.00
9,10248,90,5,1996-07-04,3,2,42,10,Singaporean Hokkien Fried Mee,20,5,32 - 1 kg pkgs.,14.00,140.00


In [64]:
total_price = od.groupby('OrderDate')['Total_Price'].sum()
# total_price

In [66]:
total_price[total_price == total_price.max()]

OrderDate
1997-01-23    15641.5
Name: Total_Price, dtype: float64

### Q4 문제를 만들고, 풀어보세요!